## Without Using Pipeline

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer # to adjust missing data
from sklearn.preprocessing import OneHotEncoder # to handle categorical data
from sklearn.preprocessing import MinMaxScaler # for scaling
from sklearn.tree import DecisionTreeClassifier # for prediction

In [14]:
df = pd.read_csv('train.csv')

In [16]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
# dropping unnecessary columns
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [22]:
df.head() # new dataframe after removal

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [20]:
# step-1: train/test/split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                    df['Survived'],
                                                    test_size=0.2,
                                                    random_state=42)

In [24]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [26]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

  *Since col: Age, Embarked contains missing values so we need to handle them first else it will create problem during model training*

In [44]:
# imputation
si_age = SimpleImputer() # for age col
si_embarked = SimpleImputer(strategy = 'most_frequent') # for embarked col

# for training data
X_train_age = si_age.fit_transform(X_train[['Age']])
X_train_embarked = si_embarked.fit_transform(X_train[['Embarked']])

# for testing data
X_test_age = si_age.transform(X_test[['Age']])
X_test_embarked = si_embarked.transform(X_test[['Embarked']])

*we can perform OHE on 'sex' and 'embarked' col as they're categorical data and their order doesn't matter they are like boolean variables*

In [50]:
# OHE: sex + embarked
ohe_sex = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
ohe_embarked = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')

X_train_sex = ohe_sex.fit_transform(X_train[['Sex']])
X_train_embarked = ohe_embarked.fit_transform(X_train_embarked)

X_test_sex = ohe_sex.transform(X_test[['Sex']])
X_test_embarked = ohe_embarked.transform(X_test_embarked)

In [56]:
# to add the modified data first we need to remove original data
X_train_remain = X_train.drop(columns=['Sex', 'Age', 'Embarked'])
X_test_remain = X_test.drop(columns=['Sex', 'Age', 'Embarked'])

In [58]:
# concatenating all the cols 
X_train_transformed = np.concatenate((X_train_remain, X_train_age, X_train_sex, X_train_embarked), axis=1) # training data
X_test_transformed = np.concatenate((X_test_remain, X_test_age, X_test_sex, X_test_embarked), axis=1) # testing data

In [64]:
X_train_transformed.shape

(712, 10)

In [66]:
# model training
clf = DecisionTreeClassifier()
clf.fit(X_train_transformed, y_train)

DecisionTreeClassifier()

In [72]:
# predicting
y_pred = clf.predict(X_test_transformed)
y_pred

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1], dtype=int64)

In [74]:
# calc accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7821229050279329

In [76]:
import pickle

In [80]:
pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('models/clf.pkl', 'wb'))

## Preprocessing

*below code is execution code which is to be run when you want to predict new value*
 - every step matters
 - ek bhi step upr niche hogyi toh error dega

In [ ]:
ohe_sex = pickle.load(open('models/ohe_sex.pkl','rb'))
ohe_embarked = pickle.load(open('models/ohe_embarked.pkl','rb'))
clf = pickle.load(open('models/clf.pkl','rb'))

In [ ]:

# Assume user input
# Pclass/gender/age/SibSp/Parch/Fare/Embarked
test_input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [ ]:
test_input

In [ ]:
test_input_sex = ohe_sex.transform(test_input[:,1].reshape(1,1))

In [ ]:
test_input_sex

In [ ]:
test_input_embarked = ohe_embarked.transform(test_input[:,-1].reshape(1,1))

In [ ]:
test_input_embarked

In [ ]:
test_input_age = test_input[:,2].reshape(1,1)

In [ ]:
test_input_transformed = np.concatenate((test_input[:,[0,3,4,5]],test_input_age,test_input_sex,test_input_embarked),axis=1)

In [ ]:
test_input_transformed.shape

In [ ]:
clf.predict(test_input_transformed)